### 1. Transform

In [52]:
import glob 
import jsonlines 
import pandas as pd 

In [53]:
data = [] 

for file in glob.glob('data_4.json'): 
    with jsonlines.open(file, 'r') as reader: 
        data += list(reader)   

In [54]:
data

[{'id': 'b9b8edff8da94c3ea064a75259e4fe61',
  'examples': [{'content': 'biệt thự 3 tầng 2 mặt tiền nam hoà xuân khu vực dân trí cao (làng đại gia) kinh doanh buôn bán sầm uất. vị trí ngay đầu đường văn tiến dũng. cách chợ hoà xuân 2 phút đi bộ. gần trường, uỷ ban phường và rất nhiều tiện ích xung quanh. pháp lý cc.',
    'seen_by': [{'annotator': 'Lanhuongqaqs@gmail.com', 'annotator_id': 1}],
    'comments': [],
    'metadata': {'': 35,
     'Mô tả': 'biệt thự 3 tầng 2 mặt tiền nam hoà xuân khu vực dân trí cao(làng đại gia)kinh doanh buôn bán sầm uất.vị trí ngay đầu đường văn tiến dũng.cách chợ hoà xuân 2 phút đi bộ.gần trường,uỷ ban phường và rất nhiều tiện ích xung quanh.pháp lý cc.'},
    'example_id': '2de1f863-6925-4620-8f23-de0cb387fbf9',
    'annotations': [{'end': 59,
      'tag': 'AN_NINH',
      'start': 48,
      'value': 'dân trí cao',
      'tag_id': 'be1e1dd7-c03c-460a-b2b9-03178093fd44',
      'correct': None,
      'reviewed': False,
      'example_id': '2de1f863-6925-4

In [55]:
lst_text = []
lst_tags = [] 

for record in data: 
    for example in record['examples']:
        text = example['content']
        lst_text.append(text)

        labels = [] 
        values = [] 

        for tag in example['annotations']:
            entity = tag['tag']
            value = text[tag['start'] : tag['end']]

            for i, v in enumerate(value.split()):
                if i == 0: 
                    labels.append(f'B-{entity}')
                else: 
                    labels.append(f'I-{entity}') 
                values.append(v)

        for i, value in enumerate(values): 
            text = text.replace(value, labels[i])

        lst = text.split()

        for idx, ent in enumerate(lst): 
            if ent[:2] not in ('B-', 'I-'):  
                lst[idx] = 'O'
        lst_tags.append(' '.join(lst))

In [56]:
df = pd.DataFrame({'text' : lst_text, 
              'tags' : lst_tags})

df.tags = df.tags.replace(',', '')

In [57]:
df

text  \
0   biệt thự 3 tầng 2 mặt tiền nam hoà xuân khu vự...   
1   nhà 3 tầng phòng khách rộng rãi, 3pn, wc, p.th...   
2   chính chủ cần bán nhà 2 tầng 2 phòng ngủ với d...   
3   khách sạn 3 sao, vị trí đắc địa trung tâm thàn...   
4   duy nhất hôm nay chốt giá bán nhanh nhà mt điệ...   
..                                                ...   
95  bán đất lô góc 2 mặt tiền đường lê thạch, cẩm ...   
96  bán đất đường phan triêm, hoà xuân, cẩm lệ, đà...   
97  nhà c4 kiệt ô tô nguyễn chánh bán nhà c4 kiệt ...   
98  bán nhà 3 tâng đường bàu năng 2 p. hòa minh q....   
99  bán nhà 3 tầng full nội thất kiệt ô tô đường t...   

                                                 tags  
0   B-LOAI_BDS I-LOAI_BDS B-SO_TANG I-SO_TANG B-VI...  
1   B-LOAI_BDS B-SO_TANG I-SO_TANG B-TIEN_ICH_KHAC...  
2   O O O B-LOAI_BDS I-LOAI_BDS B-SO_PHONG_NGU I-S...  
3   B-LOAI_BDS I-LOAI_BDS O O O O O O B-DIA_DIEM I...  
4   O O O O O O B-LOAI_BDS I-LOAI_BDS I-LOAI_BDS B...  
..                                                ...  
95  B-LOAI_BDS I-LOAI_BDS O O B-VI_TRI I-VI_TRI I-...  
96  B-LOAI_BDS I-LOAI_BDS B-DIA_CHI O I-DIA_CHI I-...  
97  I-LOAI_BDS O B-DUONG_KIET I-DUONG_KIET O I-DIA...  
98  B-LOAI_BDS B-LOAI_BDS B-SO_TANG I-SO_TANG B-DU...  
99  B-LOAI_BDS I-LOAI_BDS B-SO_PHONG_NGU I-SO_TANG...  

[100 rows x 2 columns]

### 2. Check entity error

In [58]:
check = {}

for tag in df.tags: 
    for t in tag.split(): 
        if t not in check: 
            check[t] = len(check)

In [59]:
check

{'B-LOAI_BDS': 0,
 'I-LOAI_BDS': 1,
 'B-SO_TANG': 2,
 'I-SO_TANG': 3,
 'B-VI_TRI': 4,
 'I-VI_TRI': 5,
 'O': 6,
 'B-AN_NINH': 7,
 'I-AN_NINH': 8,
 'B-KINH_DOANH': 9,
 'I-KINH_DOANH': 10,
 'I-KINH_DOANH.': 11,
 'B-DICH_VU': 12,
 'B-DICH_VU,': 13,
 'B-TIEN_ICH_KHAC': 14,
 'I-TIEN_ICH_KHAC': 15,
 'B-SO_TANGpn,': 16,
 'B-TIEN_ICH_KHAC,': 17,
 'B-CHIEU_RONGm,': 18,
 'B-DUONG_KIET': 19,
 'I-DUONG_KIET': 20,
 'I-VIA_HE,': 21,
 'B-KV_DE_XE': 22,
 'I-KV_DE_XE': 23,
 'I-DICH_VU,': 24,
 'I-DICH_VU': 25,
 'B-DAN_CU': 26,
 'I-DAN_CU': 27,
 'I-DAN_CU,': 28,
 'B-TIEM_NANG': 29,
 'I-TIEM_NANG': 30,
 'B-CHIEU_RONG': 31,
 'I-GIA': 32,
 'B-SO_PHONG_NGU': 33,
 'I-SO_PHONG_NGU': 34,
 'B-DIEN_TICH.': 35,
 'B-THOAT_NUOC': 36,
 'I-THOAT_NUOC': 37,
 'I-THOAT_NUOC.': 38,
 'B-DIA_DIEM': 39,
 'I-DIA_DIEM': 40,
 'I-DIA_DIEM,': 41,
 'B-TINH_TRANG_KD': 42,
 'I-TINH_TRANG_KD': 43,
 'I-TINH_TRANG_KD,': 44,
 'I-DUONG_KIET,': 45,
 'I-TIEM_NANG.': 46,
 'I-CHIEU_RONG0': 47,
 'B-GIA': 48,
 'I-GIA.': 49,
 'I-CHIEU_RONG': 50,

In [60]:
df[df.tags.str.contains('B-CHIEU_RONG.B-SO_PHONG_NGU')].values

array([['bán nhà 3 tầng 3 mê kiệt ô tô thái thị bôi thanh khê. đà nẵng kiệt 4.5m cách đường 70m. ô tô vào tới nhà dt 50m2 (5.2 x 9.55m) dt sd 150m2 hướng nam ( thoáng mát ) vị trí trung tâm gần các trường đại học lớn, bệnh viện dt 50m2 (5,2 x 9,55)m tầng 1 phòng khách, bếp, wc, sân để xe tầng 2 2 phòng ngủ, wc riêng mỗi phòng tầng 3 1 phòng ngủ, phòng thờ, phòng phơi đồ riêng biệt giá 3,55 tỷ zalo o987 901 827',
        'B-LOAI_BDS I-LOAI_BDS B-SO_TANG I-SO_TANG B-SO_TANG O O O O O O O O O O O O O O O O O O O O I-LOAI_BDS O O B-CHIEU_RONG.B-SO_PHONG_NGU O B-CHIEU_RONG) O O O B-HUONG_NHA I-HUONG_NHA O O O O O O B-DIA_DIEM I-DIA_DIEM O O B-DICH_VU B-DICH_VU I-DICH_VU O B-DICH_VU I-DICH_VU O O B-CHIEU_RONG,B-SO_PHONG_NGU O B-CHIEU_DAI I-SO_TANG O I-SO_PHONG_NGU I-TIEN_ICH_KHAC, B-TIEN_ICH_KHAC, O B-KV_DE_XE I-KV_DE_XE I-KV_DE_XE I-SO_TANG B-SO_PHONG_NGU B-SO_PHONG_NGU I-SO_PHONG_NGU I-SO_PHONG_NGU, O O O I-SO_PHONG_NGU I-SO_TANG B-SO_TANG O I-SO_PHONG_NGU I-SO_PHONG_NGU, I-SO_PHONG_NGU I-

# 3. Check value - entity

In [61]:
def compare_chars_with_labels(text, tags):
    tokens = text.split()
    label_list = tags.split()

    # Kiểm tra độ dài của tokens và labels
    if len(tokens) != len(label_list):
        raise ValueError("Số lượng tokens không khớp với số lượng labels")

    for token, label in zip(tokens, label_list):
        print(f"{token} - {label}")

In [62]:
for index, row in df.iterrows():
    print(f"Row {index + 1}: ", df.text[index])
    compare_chars_with_labels(row['text'], row['tags'])
    print("\n")

Row 1:  biệt thự 3 tầng 2 mặt tiền nam hoà xuân khu vực dân trí cao (làng đại gia) kinh doanh buôn bán sầm uất. vị trí ngay đầu đường văn tiến dũng. cách chợ hoà xuân 2 phút đi bộ. gần trường, uỷ ban phường và rất nhiều tiện ích xung quanh. pháp lý cc.
biệt - B-LOAI_BDS
thự - I-LOAI_BDS
3 - B-SO_TANG
tầng - I-SO_TANG
2 - B-VI_TRI
mặt - I-VI_TRI
tiền - I-VI_TRI
nam - O
hoà - O
xuân - O
khu - O
vực - O
dân - B-AN_NINH
trí - I-AN_NINH
cao - I-AN_NINH
(làng - O
đại - O
gia) - O
kinh - O
doanh - O
buôn - B-KINH_DOANH
bán - I-KINH_DOANH
sầm - I-KINH_DOANH
uất. - I-KINH_DOANH.
vị - O
trí - I-AN_NINH
ngay - O
đầu - O
đường - O
văn - O
tiến - O
dũng. - O
cách - O
chợ - B-DICH_VU
hoà - O
xuân - O
2 - B-VI_TRI
phút - O
đi - O
bộ. - O
gần - O
trường, - B-DICH_VU,
uỷ - O
ban - O
phường - O
và - O
rất - O
nhiều - O
tiện - O
ích - O
xung - O
quanh. - O
pháp - O
lý - O
cc. - O


Row 2:  nhà 3 tầng phòng khách rộng rãi, 3pn, wc, p.thờ, sân phơi mặt tiền 5m, đường 8m ,lề 4m, đỗ ô tô vivu khu vực giao th

đh - O
ngoại - O
ngữ - O
tầm - O
3 - O
phút, - O
trường - B-DICH_VU
đh - O
duy - O
tân - O
tầm - O
3 - O
phút, - O
cách - O
sân - B-DICH_VU
bay - I-DICH_VU
tầm - O
6 - O
phút, - O
cách - O
bệnh - B-DICH_VU
viện - I-DICH_VU
đa - O
khoa - O
tầm - O
4 - O
phút, - O
cách - O
công - B-DICH_VU
viên - I-DICH_VU
tầm - O
3 - O
phút, - O
gần - O
siêu - B-DICH_VU
thị - I-DICH_VU
go, - O
gần - O
chợ - B-DICH_VU
cồn. - O
tiện - O
ích - O
xung - O
quanh - O
chỉ - O
vài - O
bước - O
chân. - O
đang - B-TINH_TRANG_KD
có - I-TINH_TRANG_KD
hd - I-TINH_TRANG_KD
cho - I-TINH_TRANG_KD
thuê - I-TINH_TRANG_KD
kinh - I-TINH_TRANG_KD
doanh - I-TINH_TRANG_KD
thời - I-TINH_TRANG_KD
trang. - I-TINH_TRANG_KD.
lh - O
mr - O
nguyên - O
giá - O
tốt - O
cho - I-TINH_TRANG_KD
nhà - O
đầu - O
tư. - O


Row 22:  với vị trí đắc địa tọa lạc trên đại lộ võ nguyên giáp , diện tích 500m2 đất ở lâu dài (odt) , quy mô 100 phòng khách sạn kinh doanh, nhà hàng ăn sáng , phòng hội nghị , hồ bơi đầy đủ dv đi kèm giá chuyển nhượng vô

trung - O
lượng - O
khu - O
vip - I-NOI_THATp
thuộc - O
hoà - O
xuân, - O
cẩm - O
lệ - O
đường - B-DUONG_KIET
7m5, - I-DUONG_KIETI-NOI_THAT
song - O
song - O
với - O
nguyễn - O
phước - O
lan - O
diện - O
tích - O
100m2 - B-DIEN_TICH
(ngang - O
5m - I-CHIEU_RONG
), - O
dtsd - O
279 - O
m2 - O
hướng - B-HUONG_NHA
đông - I-HUONG_NHA
bắc - I-HUONG_NHA
nhà - I-LOAI_BDS
3 - B-SO_PHONG_NGU
tầng - I-SO_TANG
full - B-NOI_THAT
nội - I-NOI_THAT
thất - I-NOI_THAT
ti - I-NOI_THAT
vi - I-NOI_THAT
, - I-NOI_THAT
tủ - I-NOI_THAT
lạnh - I-NOI_THAT
, - I-NOI_THAT
đầy - I-PHAP_LY
đủ - I-PHAP_LY
tiện - I-NOI_THATện
nghi - I-NOI_THAT
... - O
công - O
năng - O
nhà - I-LOAI_BDS
gồm - O
3 - B-SO_PHONG_NGU
phòng - I-SO_PHONG_NGU
ngủ, - I-SO_PHONG_NGUI-NOI_THAT
4wc, - B-SO_TOILETI-NOI_THAT
sân - B-TIEN_ICH_KHAC
phơi - I-TIEN_ICH_KHAC
tủ - I-NOI_THAT
bếp - B-TIEN_ICH_KHAC
cao - I-AN_NINH
cấp - O
vị - O
trí - I-AN_NINH
đẹp, - O
khu - O
dân - B-AN_NINH
trí - I-AN_NINH
cao, - I-AN_NINHI-NOI_THAT
an - B-AN_NINH
ninh

34,9 - O
tỷ - I-GIA
( - O
lvcc - O
) - O
liên - O
hệ - O
mr.nhật - O
để - I-KV_DE_XE
được - O
tư - O
vấn - O
và - O
xem - I-KV_DE_XEm
thực - O
tế - O
toà - B-LOAI_BDS
căn - I-LOAI_BDS
hộ - I-LOAI_BDS
36land - O
nhận - O
ký - O
gửi - O
bđs - O
đà - O
nẵng - O


Row 53:  nhà 4 tầng mặt tiền lý tự trọng. sát đường bạch đằng. nôi thất đẹp sang xịn mịn. ngang 3.5 mt 1 khách bếp t2 ngủ wc t3 ngủ wc t4 ngủ giặt đầy đủ công năng. công năng gồm 3 điều hoà (loại xịn mới mua dien may xanh 2 tháng) 3 quạt trần ( mới mua 20t) 1 bếp (moi mua 20t) tủ giường trang điểm ...v...v... mới đóng toàn loại tốt 1 nc nóng 1 máy giặt 1 máy sấy 1 tủ lạnh (tất cả đêu mới mua 2 tháng còn bảo hành) giá bán nhanh hcc tiếp khách thiện chí tìm kiếm theo từ khóa bán nhà mặt tiền hải châu, bán nhà mặt tiền gần sông hàn hải châu, bán nhà mặt tiền lý tự trọng hải châu
nhà - I-LOAI_BDS
4 - B-SO_TANG
tầng - I-SO_TANG
mặt - B-VI_TRI
tiền - I-VI_TRI
lý - O
tự - O
trọng. - O
sát - O
đường - O
bạch - O
đằng. - O
nôi - B-NOI_THA

shophouse - O
và - O
biệt - B-LOAI_BDS
thự - I-LOAI_BDS
diện - O
tích - O
từ - O
140 - O
400m². - B-DIEN_TICH².
thông - O
tin - O
chi - O
tiết - O
về - O
giá - O
và - O
chính - O
sách - O
bán - O
hàng - I-DICH_VU
căn - O
hộ - O
tòa - O
p2 - O
view - B-VIEW
sông - B-DICH_VU
hàn - I-VIEW
hoặc - O
shophouse - O
quý - O
khách - B-TIEN_ICH_KHAC
vui - O
lòng - O
liên - O
hệ - O


Row 65:  chủ chuyển chổ ở cần bán 1 căn gốc fhome tại 16 lý thường kiệt, quận hải châu, tp đà nẵng diện tích 63m2 gồm 2pn 2wc sổ hồng sở hữu lâu dài, công chứng ngay để lại toàn bộ nội thất view biển view thành phố giá bán 2,55 tỷ liên hệ xem nhà 0903123157 tuyết nhi ( zalo phone viber)
chủ - O
chuyển - O
chổ - O
ở - O
cần - O
bán - O
1 - O
căn - O
gốc - O
fhome - O
tại - O
16 - B-DIA_CHI
lý - I-DIA_CHI
thường - I-DIA_CHI
kiệt, - I-DIA_CHI
quận - I-DIA_CHI
hải - I-DIA_CHI
châu, - I-DIA_CHI
tp - I-DIA_CHI
đà - I-DIA_CHI
nẵng - I-DIA_CHI
diện - O
tích - O
63m2 - B-DIEN_TICH
gồm - O
2pn - B-SO_PHONG_NGU
2wc - B-SO_TOIL

nam - I-HUONG_NHA
mát - O
mẻ - O
từ - O
villa - B-LOAI_BDS
ra - O
biển - I-VIEW
chưa - O
tới - O
100 - B-KV_DE_XE00
mét - O
, - O
phía - O
sau - O
nhà - B-LOAI_BDS
cũng - O
có - O
view - B-VIEW
biển - I-VIEW
và - O
sân - B-SAN_VUON
golf - O
thiết - O
kế - O
công - O
năng - O
villa - B-LOAI_BDS
2 - I-KV_DE_XE
tầng - I-SO_TANG
1 - B-KV_DE_XE
phòng - I-SO_TOILET
khách, - I-TIEN_ICH_KHAC,
4 - B-SO_TOILET
phòng - I-SO_TOILET
ngủ - I-SO_PHONG_NGU
lớn, - I-KV_DE_XE,
4 - B-SO_TOILET
phòng - I-SO_TOILET
wc - I-SO_TOILET
nam - I-HUONG_NHA
nữ. - O
ngoại - O
thất - O
full - B-NOI_THAT
nhập - I-NOI_THAT
ngoại, - I-NOI_THAT
đá - I-NOI_THAT
marble - I-NOI_THAT
cao - I-NOI_THAT
cấp - I-NOI_THAT
tây - I-NOI_THAT
ban - I-NOI_THAT
nha - I-NOI_THAT
nhập. - I-NOI_THAT.
có - O
hồ - B-TIEN_ICH_KHAC
bơi - I-TIEN_ICH_KHAC
riêng - O
view - B-VIEW
trực - O
tiếp - O
mặt - O
biển. - I-VIEW.
1 - B-KV_DE_XE
gara - I-KV_DE_XE
để - I-KV_DE_XE
2 - I-KV_DE_XE
ô - I-KV_DE_XE
tô - O
lớn. - I-KV_DE_XE.
1 - B-KV_DE_XE
sân -

Row 95:  bán nhà 3 tầng 3 mê kiệt ô tô thái thị bôi thanh khê. đà nẵng kiệt 4.5m cách đường 70m. ô tô vào tới nhà dt 50m2 (5.2 x 9.55m) dt sd 150m2 hướng nam ( thoáng mát ) vị trí trung tâm gần các trường đại học lớn, bệnh viện dt 50m2 (5,2 x 9,55)m tầng 1 phòng khách, bếp, wc, sân để xe tầng 2 2 phòng ngủ, wc riêng mỗi phòng tầng 3 1 phòng ngủ, phòng thờ, phòng phơi đồ riêng biệt giá 3,55 tỷ zalo o987 901 827
bán - B-LOAI_BDS
nhà - I-LOAI_BDS
3 - B-SO_TANG
tầng - I-SO_TANG
3 - B-SO_TANG
mê - O
kiệt - O
ô - O
tô - O
thái - O
thị - O
bôi - O
thanh - O
khê. - O
đà - O
nẵng - O
kiệt - O
4.5m - O
cách - O
đường - O
70m. - O
ô - O
tô - O
vào - O
tới - O
nhà - I-LOAI_BDS
dt - O
50m2 - O
(5.2 - B-CHIEU_RONG.B-SO_PHONG_NGU
x - O
9.55m) - B-CHIEU_RONG)
dt - O
sd - O
150m2 - O
hướng - B-HUONG_NHA
nam - I-HUONG_NHA
( - O
thoáng - O
mát - O
) - O
vị - O
trí - O
trung - B-DIA_DIEM
tâm - I-DIA_DIEM
gần - O
các - O
trường - B-DICH_VU
đại - B-DICH_VU
học - I-DICH_VU
lớn, - O
bệnh - B-DICH_VU
viện - I-

# 4. Save df

In [ ]:
# df.to_csv()